# CREDIT CARD APPROVAL PROJECT

This notebook presents a complete machine learning project to predict **credit card approval** using client application data and historical credit behavior.

We'll work with two datasets:

- `application_record.csv`: Contains demographic and financial attributes for each client (one row per client).
- `credit_record.csv`: Contains monthly credit status history per client (multiple rows per client).

The goal is to build a **model** that predicts whether a client should be approved for a credit card, based on their profile and past credit behavior.

### Key steps in this notebook:

1. **Load and inspect the data**
2. **Clean and merge the datasets**
3. **Define a meaningful target variable** (late payment history as a proxy for credit risk for example)
4. **Explore and visualize the data**
5. **Engineer useful features**
6. **Train and evaluate classification models**
7. **Interpret model results and feature importance**

This project will highlight best practices in real-world data science, including dealing with missing values, imbalanced classes, and model evaluation beyond accuracy.  
**NOTE:** We were told on Kaggle that `"unbalance data problem is a big problem in this task."` so we have to check the balance...

Let's get to it.

In [17]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('future.no_silent_downcasting', True)

In [18]:
df_app = pd.read_csv("data/application_record.csv")
df_credit = pd.read_csv("data/credit_record.csv")

In [19]:
# Taking a look at the datasets
df_app

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0


In [20]:
df_credit

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [21]:
df_app.isna().sum()

ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        134203
CNT_FAM_MEMBERS             0
dtype: int64

In [22]:
df_credit.isna().sum()

ID                0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

## DATA HANDLING

In [23]:
# Let's get to the data handling, first we're going to focus on the df_credit dataframe
df_credit

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [24]:
# We're going to turn data in the "status" column and with that data we're going to build our target column
df_credit.STATUS.value_counts()

STATUS
C    442031
0    383120
X    209230
1     11090
5      1693
2       868
3       320
4       223
Name: count, dtype: int64

STATUS: Credit status, where:
* `0`: 1-29 days past due.
* `1`: 30-59 days past due.
* `2`: 60-89 days past due.
* `3`: 90-119 days past due.
* `4`: 120-149 days past due.
* `5`: 150+ days past due or written off.
* `C`: Completed (paid off).
* `X`: No loan for the month.

So we have to determine who is a bad and who is a good client, we're going to say that if the status is over or equal to 2, then it's a bad one


In [25]:
bad_clients = df_credit[df_credit['STATUS'].isin(["2","3", "4", "5"])]
bad_ids = bad_clients.ID.unique()

In [26]:
# Now let's create a LABELS dataframe to merge into the application dataframe and know if a client is bad or not
df_labels = pd.DataFrame({"ID": df_credit.ID.unique()})
df_labels["LABEL"] = df_labels.ID.apply(lambda x: 1 if x in bad_ids else 0) # Applies 1 if the client is bad and 0 if its good
df_labels

,ID,LABEL
0,5001711,0
1,5001712,0
2,5001713,0
3,5001714,0
4,5001715,0
...,...,...
45980,5150482,0
45981,5150483,0
45982,5150484,0
45983,5150485,0


In [27]:
df_labels.LABEL.value_counts(normalize=True)

LABEL
0    0.985495
1    0.014505
Name: proportion, dtype: float64

So the proportion here is horrendous, so we have to manage this as soon as posible...

But first, lets merge the dataframes and call it `df`

In [28]:
df = pd.merge(df_app, df_labels, on= "ID", how="inner")
df

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,LABEL
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5149828,M,Y,Y,0,315000.0,Working,Secondary / secondary special,Married,House / apartment,-17348,-2420,1,0,0,0,Managers,2.0,1
36453,5149834,F,N,Y,0,157500.0,Commercial associate,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0,1
36454,5149838,F,N,Y,0,157500.0,Pensioner,Higher education,Married,House / apartment,-12387,-1325,1,0,1,1,Medicine staff,2.0,1
36455,5150049,F,N,Y,0,283500.0,Working,Secondary / secondary special,Married,House / apartment,-17958,-655,1,0,0,0,Sales staff,2.0,1


## HANDLING UNBALANCED DATA


we're going to use SMOTE to generate synthetic samples for the 'bad' class. And then we're going to encode the data...

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define columns
binary_cols = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL']
categorical_cols = ['NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE']
numerical_cols = ['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'CNT_FAM_MEMBERS']

# Fill missing values on occupation_type
df.OCCUPATION_TYPE = df.OCCUPATION_TYPE.fillna("Unknown")

# Create an age and a years working column
df["AGE"] = np.abs(df.DAYS_BIRTH) // 365
df["YEARS_WORKING"] = df.DAYS_EMPLOYED.apply(lambda x: -x // 365 if x < 0 else 0) # to make the 365245 flag equal to 0
df.drop(["DAYS_BIRTH", "DAYS_EMPLOYED"], axis=1, inplace=True) # drop the original ones

# Define X & y and target
X = df.drop(["ID", "LABEL"], axis=1)
y = df["LABEL"]

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('bin', OneHotEncoder(drop='first', handle_unknown='ignore'), binary_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
    ]
)

# BUILD, TRAIN AND TEST MODEL

In [ ]:
# We're going to test various models to see which one is the best performer with the data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


/Users/ignaciomelinc/Documents/ML-DS/credit-card-approval-project/my_env/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [1, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/ignaciomelinc/Documents/ML-DS/credit-card-approval-project/my_env/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [1, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/ignaciomelinc/Documents/ML-DS/credit-card-approval-project/my_env/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 8] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/ignaciomelinc/Documents/ML-DS/credit-card-approval-project/my_env/lib/python3.12/site-packages/sklearn/preprocessing/

MODEL: Logistic Regression 
-- ROC_AUC_MEAN: 0.5438527447786843 
-- SCORES: [0.54402367 0.54557104 0.55152983 0.45545866 0.62268052]


/Users/ignaciomelinc/Documents/ML-DS/credit-card-approval-project/my_env/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [1, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/ignaciomelinc/Documents/ML-DS/credit-card-approval-project/my_env/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 8] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/ignaciomelinc/Documents/ML-DS/credit-card-approval-project/my_env/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 8] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/ignaciomelinc/Documents/ML-DS/credit-card-approval-project/my_env/lib/python3.12/site-packages/sklearn/preproce

MODEL: Random Forest 
-- ROC_AUC_MEAN: 0.5304592055768331 
-- SCORES: [0.52385157 0.49283498 0.54524229 0.50210001 0.58826719]
